# Householder and Givens Tools

The plan is to implement a number of things in pure Julia to provide access to the AD tools.

1. Single step in place two-sided Householder reduction
1. XXX

## Householder

## Householder Start

In [87]:
using LinearAlgebra
m=5;
A = rand(m,m); H=copy(A)
v=zeros(m)
# 1st column
i=1
v=zeros(m)
v[i+1:m] = H[i+1:m,i]
v[i+1]-=norm(v)
v /= norm(v)
H -= 2.0*(H*v)*v'
H -= 2.0*v*(H'*v)'
# 2nd column
i=2
v=zeros(m)
v[i+1:m] = H[i+1:m,i]
v[i+1]-=norm(v)
v /= norm(v)
H -= 2.0*(H*v)*v'
H -= 2.0*v*(H'*v)'
# 3rd column 
# with mutation and fixed storage for v as an m vector
i=3
#v=0*v
v[i+1:m] = H[i+1:m,i]
v[i+1]-=norm(v[i+1:m])
v[i+1:m] /= norm(v[i+1:m])
H[i+1:m,i:m] -= 2.0*v[i+1:m]*(H[i+1:m,i:m]'*v[i+1:m])'
H[1:m,i+1:m] -= 2.0*(H[1:m,i+1:m]*v[i+1:m])*v[i+1:m]'
# Check
print(norm(eigen(A).values-eigen(H).values))
H

3.589387267483526e-15

5×5 Matrix{Float64}:
  0.0134093     1.11518      0.194692      0.120225  -0.461721
  1.08501       1.81417      0.88617      -0.505577   0.0925293
  1.54373e-17   1.32182      0.698382      0.747266  -0.00751425
  7.72658e-17  -1.66533e-16  0.12028      -0.216856   0.224815
 -7.82153e-17  -3.88578e-16  1.38778e-17   0.191219   0.293834

## Loop Script

In [88]:
using LinearAlgebra
m=5;
A = rand(m,m); H=copy(A)
v=zeros(m)
for i in 1:m-2
    v[i+1:m] = H[i+1:m,i]
    v[i+1]-=norm(v[i+1:m])
    v[i+1:m] /= norm(v[i+1:m])
    H[i+1:m,i:m] -= 2.0*v[i+1:m]*(H[i+1:m,i:m]'*v[i+1:m])'
    H[1:m,i+1:m] -= 2.0*(H[1:m,i+1:m]*v[i+1:m])*v[i+1:m]'
end
# Check
print(norm(eigen(A).values-eigen(H).values))
H

2.16724662841394e-15

5×5 Matrix{Float64}:
  0.826014     0.814989     0.501048      0.345439    0.0581925
  1.37216      1.40698      0.425821      1.11297    -0.158519
 -5.55112e-17  1.17193      0.31592       0.241514   -0.534538
  0.0          0.0          0.606833     -0.0662567  -0.509651
 -2.22045e-16  1.11022e-16  2.22045e-16  -0.416923    0.0215552

## In place function with workspace.

The fixed size workspace is called v. It is a full size m vector. 

Note: I can use a smaller "end" value "k" for the bulge structure. 

In [54]:
function HessPosDown(H,v,i,k)
    # Downwards lower triangular Householder variant with positive subdiagonal. 
    # bottom subdiagonal sign can be -
    v[i+1:k] = H[i+1:k,i]
    v[i+1]-=norm(v[i+1:k])
    v[i+1:k] /= norm(v[i+1:k])
    H[i+1:k,i:k] -= 2.0*v[i+1:k]*(H[i+1:k,i:k]'*v[i+1:k])'
    H[1:k,i+1:k] -= 2.0*(H[1:k,i+1:k]*v[i+1:k])*v[i+1:k]'
end

function HessPosUp(H,v,i,k)
    # Upwards lower triangular Householder variant with positive subdiagonal. 
    # top subdiagonal sign can be -
    v[i+1:k] = H[i,i+1:k]
    v[i+1]-=norm(v[i+1:k])
    v[i+1:k] /= norm(v[i+1:k])
    H[i+1:k,i:k] -= 2.0*v[i+1:k]*(H[i+1:k,i:k]'*v[i+1:k])'
    H[1:k,i+1:k] -= 2.0*(H[1:k,i+1:k]*v[i+1:k])*v[i+1:k]'
end

HessPosUp (generic function with 1 method)

In [76]:
using LinearAlgebra
m=4
A=rand(m,m); HD = copy(A); HU = copy(A); v=zeros(m)
# Downwards variant
for i in 1:1:m-2
    HessPosDown(HD,v,i,m)
end
# Fix ultimate subdiagonal sign if needed 
if HD[m,m-1]<0 
        HD[m,1:m]*=-1;HD[1:m,m]*=-1 
end
triu!(HD,-1)
println("eval diff down: ",norm(eigen(HD).values - eigen(A).values))

In [78]:
using LinearAlgebra
m=4
A=rand(m,m); HD = copy(A); HU = copy(A); v=zeros(m)
# Upwards variant
for i in m-1:m-1
    HessPosDown(HU,v,i,m)
end
HU

4×4 Matrix{Float64}:
 0.455758  0.548601    0.861171  NaN
 0.99836   0.606171    0.705193  NaN
 0.823653  0.770931    0.434828  NaN
 0.12408   0.505592  NaN         NaN

In [72]:
println("eval diff down: ",norm(eigen(HD).values - eigen(A).values))
println("eval diff up: ",norm(eigen(HU).values - eigen(A).values))
println(norm(A-HU))

eval diff down: 5.453105385130469e-16
eval diff up: 0.0
0.0


In [71]:
HU

4×4 Matrix{Float64}:
 0.572427  0.468067    0.641378  NaN
 0.902801  0.717841    0.575609  NaN
 0.371599  0.351586    0.899509  NaN
 0.829578  0.161262  NaN         NaN

In [30]:
using LinearAlgebra, PlotlyJS
m=7;
A = rand(m,m); H=copy(A); v=zeros(m);
for i in 1:m-2
    Hess1(H,v,i,m)
end
H=triu(H,-1)
println("eval diff: ",norm(eigen(H).values -eigen(A).values))
H

eval diff: 4.994769877343149e-15


7×7 Matrix{Float64}:
 0.793949  1.74034  0.88086   0.0583797  -0.111496   -0.327828  -0.0778597
 1.22369   2.15831  0.541881  0.881448   -0.323069    0.115179   0.213928
 0.0       1.69135  0.801177  0.24762    -0.212174    0.110208   0.254577
 0.0       0.0      0.438226  0.278147    0.417774    0.133067   0.0961536
 0.0       0.0      0.0       0.658582    0.0424085   0.194908   0.303863
 0.0       0.0      0.0       0.0         0.290043    0.170293  -0.469382
 0.0       0.0      0.0       0.0         0.0        -0.532372   0.210992

# Multiple Shifts

I need to build the first column of the product 

$$
(H-\mu_1 I)(H-\mu_2 I) ... (H-\mu_p I)
$$

I am going to input a list of shifts "mu", the Hessenberg matrix H and a workspace "col" for the output. I also need to last row but we will do that later. 

In [16]:
function FirstColFromShifts(H, mu)
    # H is a Hessenberg matrix and mus is a list of p shifts 
    # output is in the first p+1 entries of col 
    p = size(mu)[1]
    col = H[:,1]; col[1]-=mu[1]
    for i in 2:p
        col =H*col-mu[i]*col
    end
    col
end

function LastRowFromShifts(H, mu)
    # H is a Hessenberg matrix and mus is a list of p shifts 
    # output is in the last p+1 entries of row 
    p = size(mu)[1]
    m=size(H)[1]
    row = H[m,:]; row[m]-=mu[1]
    for i in 2:p
        row =H'*row-mu[i]*row
    end
    row
end

LastRowFromShifts (generic function with 1 method)

In [17]:
using LinearAlgebra
(m,p)=(5,2) 
H=triu(rand(m,m),-1) 
col=zeros(m) 
mu=zeros(p); mu[1]=2; mu[2]=3 
FirstCol = FirstColFromShifts(H,mu) 
LastRow = LastRowFromShifts(H,mu) 
ShiftProd=H-mu[1]*I
for i in 2:p
    ShiftProd=ShiftProd*(H-mu[i]*I)
end
ShiftProd

5×5 Matrix{Float64}:
  4.98381  -3.39309   -0.103377  -2.68368   -1.40058
 -1.34573   4.45099   -2.36094   -0.589802  -1.22046
  0.117    -1.46573    3.60625   -0.105566  -1.4393
  0.0       0.168866  -1.92965    5.68155   -1.55354
  0.0       0.0        0.327647  -3.26221    4.6509

In [20]:
[FirstCol LastRow]'

2×5 adjoint(::Matrix{Float64}) with eltype Float64:
 4.98381  -1.34573  0.117      0.0      0.0
 0.0       0.0      0.327647  -3.26221  4.6509

LoadError: UndefVarError: a11 not defined